# 유튜브 비디오 데이터 크롤링

본 Jupyter Notebook은 유튜브 데이터를 수집하기 위해 사용됩니다.  
하지만 현재 유튜브에서는 크롤링을 막아 놓아, 데이터를 수집하는 것이 불가능합니다.  
유튜브의 데이터 접근 방식이 변경됨에 따라 기존의 크롤링 기법으로는 원하는 정보를 얻기 어려운 상황입니다.

## 주의사항

### 1. 크롤링 제한
- 현재 유튜브에서는 크롤링을 막아 놓아, 데이터를 수집하는 것이 불가능합니다.  
유튜브의 데이터 접근 방식이 변경됨에 따라 기존의 크롤링 기법으로는 원하는 정보를 얻기 어려운 상황입니다.

### 2. 회사 보안 정책
- 이 코드 예시는 교육 목적으로 제공된 것이며, 실제 운영 코드와 다를 수 있습니다.  
회사의 보안 정책에 따라 실제 운영 환경에서는 특정 보안 조치가 필요할 수 있습니다.  
따라서 이 코드를 직접 운영 환경에서 사용하기 전, 반드시 내부 규정 및 보안 절차를 확인해야 합니다.

---

### 추가 정보
- **유튜브 데이터 API**: 필요한 정보를 수집하기 위해 [YouTube Data API](https://developers.google.com/youtube/v3) 사용을 고려해 보세요.
- **법적 고지**: 웹 크롤링은 웹사이트의 이용 약관을 준수해야 합니다.


In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [ ]:
# 크롬 드라이버 설정 (크롬 드라이버 경로 수정 필요)
chrome_driver_path = 'path/to/chromedriver'  # 크롬 드라이버 경로를 설정하세요.
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
def get_video_data(video_url):
    """
    주어진 유튜브 비디오 URL에서 비디오 정보를 수집하는 함수입니다.

    Args:
        video_url (str): 비디오의 URL

    Returns:
        dict: 비디오 정보가 포함된 딕셔너리
    """
    # 유튜브 비디오 페이지 열기
    driver.get(video_url)
    time.sleep(2)  # 페이지 로드 대기

    # 페이지의 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 비디오 정보 수집을 위한 딕셔너리 초기화
    video_data = {}

    # 비디오 ID 추출
    video_data['video_id'] = video_url.split('v=')[-1]

    # 채널 ID 추출
    channel_element = soup.find('a', {'class': 'yt-simple-endpoint style-scope yt-formatted-string'})
    if channel_element:
        video_data['channel_id'] = channel_element['href'].split('/')[-1]  # URL에서 채널 ID 추출

    # 비디오 제목 추출
    title_element = soup.find('h1', {'class': 'title style-scope ytd-video-primary-info-renderer'})
    video_data['video_title'] = title_element.text if title_element else None  # 제목이 없을 경우 None

    # 비디오 설명 추출
    description_element = soup.find('yt-formatted-string', {'class': 'content style-scope ytd-video-secondary-info-renderer'})
    video_data['video_description'] = description_element.text if description_element else None  # 설명이 없을 경우 None

    # 비디오 태그 추출
    tag_elements = soup.find_all('meta', {'name': 'keywords'})
    video_data['video_tags'] = tag_elements[0]['content'].split(',') if tag_elements else []  # 태그가 없을 경우 빈 리스트

    # 비디오 길이 추출
    duration_element = soup.find('span', class_='ytp-time-duration')
    video_data['video_duration'] = duration_element.text if duration_element else None  # 길이가 없을 경우 None

    # 업로드일 추출
    published_element = soup.find('div', {'id': 'date'})
    video_data['video_published'] = published_element.text.strip() if published_element else None  # 업로드일이 없을 경우 None

    # 기타 정보는 제공되지 않음
    video_data['video_category'] = None
    video_data['video_info_card'] = None
    video_data['video_with_ads'] = None
    video_data['video_end_screen'] = None
    video_data['video_cluster'] = None

    return video_data

In [ ]:
# 비디오 URL 리스트
video_urls = [
    'https://www.youtube.com/watch?v=VIDEO_ID_1',  # 여기에 실제 비디오 URL을 입력하세요.
    'https://www.youtube.com/watch?v=VIDEO_ID_2'   # 여기에 실제 비디오 URL을 입력하세요.
]

In [ ]:
# 비디오 데이터 수집을 위한 리스트 초기화
video_data_list = []

# 각 비디오 URL에 대해 데이터 수집
for url in video_urls:
    video_data = get_video_data(url)  # 비디오 데이터 수집
    video_data_list.append(video_data)  # 수집한 데이터 리스트에 추가


In [ ]:
# 수집된 데이터를 DataFrame으로 변환
df = pd.DataFrame(video_data_list)

# DataFrame을 CSV 파일로 저장
df.to_csv('../data/video_sample.csv', index=False, encoding='utf-8-sig')


In [ ]:
# 드라이버 종료
driver.quit()